# ⛓ Chain 

In [ ]:
#| default_exp chains

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import asyncio, web3, os
from functools import wraps, reduce
from typing import List, TypeVar, Callable, Optional, Tuple, Dict
from fastcore.utils import patch
from web3 import Web3, HTTPProvider, AsyncWeb3, AsyncHTTPProvider, Account
from web3.eth.async_eth import AsyncContract
from web3.eth import Contract
from sugar.config import ChainSettings, make_op_chain_settings, make_base_chain_settings
from sugar.helpers import normalize_address, MAX_UINT256, float_to_uint256, apply_slippage, get_future_timestamp
from sugar.abi import sugar, slipstream, price_oracle, router
from sugar.token import Token
from sugar.pool import LiquidityPool
from sugar.price import Price
from sugar.deposit import Deposit
from sugar.helpers import ADDRESS_ZERO, chunk, normalize_address

## Chain implementation 

In [ ]:
#| export

T = TypeVar('T')

def require_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    def wrapper(self: 'CommonChain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return f(self, *args, **kwargs)
    return wrapper

def require_async_context(f: Callable[..., T]) -> Callable[..., T]:
    @wraps(f)
    async def wrapper(self: 'CommonChain', *args, **kwargs) -> T:
        if not self._in_context: raise RuntimeError("Chain methods can only be accessed within 'async with' block")
        return await f(self, *args, **kwargs)
    return wrapper


class CommonChain:
    @property
    def account(self) -> Account: return self.web3.eth.account.from_key(os.getenv("SUGAR_PK"))

    def __init__(self, settings: ChainSettings):
        self.settings, self._in_context = settings, False
    
    def prepare_tokens(self, tokens: List[Tuple], listed_only: bool) -> List[Token]:
        native = Token.make_native_token(self.settings.native_token_symbol, self.settings.wrapped_native_token_addr, self.settings.native_token_decimals)
        ts = list(map(lambda t: Token.from_tuple(t), tokens))
        return [native] + (list(filter(lambda t: t.listed, ts)) if listed_only else ts)
    
    def _prepare_prices(self, tokens: List[Token], rates: List[int]) -> Dict[str, int]:
        # token_address => normalized rate
        result = {}
        # rates are returned multiplied by eth decimals + the difference in decimals to eth
        # we want them all normalized to 18 decimals
        for cnt, rate in enumerate(rates):
            t, eth_decimals = tokens[cnt], self.settings.native_token_decimals
            if t.decimals == eth_decimals: nr = rate
            elif t.decimals < eth_decimals: nr = rate // (10 ** (eth_decimals - t.decimals))
            else: nr = rate * (10 ** (t.decimals - eth_decimals))
            result[t.token_address] = nr
        return result

    def prepare_prices(self, tokens: List[Token], prices: List[int]) -> List[Price]:
        """Get prices for tokens in target stable token"""
        eth_decimals = self.settings.native_token_decimals
        # all rates in EHT: token => rate
        rates_in_eth = self._prepare_prices(tokens, prices)
        eth_rate, usd_rate = rates_in_eth[self.settings.native_token_symbol], rates_in_eth[self.settings.stable_token_addr]
        # this gives us the price of 1 eth in usd with 18 decimals precision
        eth_usd_price = (eth_rate * 10 ** eth_decimals) // usd_rate
        # finally convert to prices in terms of stable
        return [Price(token=t, price=(rates_in_eth[t.token_address] * eth_usd_price // 10 ** eth_decimals) / 10 ** eth_decimals) for t in tokens]
    
    def prepare_pools(self, pools: List[Tuple], tokens: List[Token], prices: List[Price]) -> List[LiquidityPool]:
        tokens, prices = {t.token_address: t for t in tokens}, {price.token.token_address: price for price in prices}
        return list(filter(lambda p: p is not None, map(lambda p: LiquidityPool.from_tuple(p, tokens, prices), pools)))

### Aync chain

In [ ]:
#| export

class AsyncChain(CommonChain):
    web3: AsyncWeb3
    sugar: AsyncContract
    router: AsyncContract

    async def __aenter__(self):
        """Async context manager entry"""
        self._in_context = True
        self.web3 = AsyncWeb3(AsyncHTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=sugar)
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=slipstream)
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=price_oracle)
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=router)
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        self._in_context = False
        await self.web3.provider.disconnect()
        return None

    @require_async_context
    async def get_all_tokens(self, listed_only: bool = True) -> List[Token]:
        tokens = await self.sugar.functions.tokens(self.settings.pagination_limit, 0, ADDRESS_ZERO, []).call()
        return self.prepare_tokens(tokens, listed_only)
    
    # @cache_in_seconds(ORACLE_PRICES_CACHE_MINUTES * 60)
    async def _get_prices(self, tokens: Tuple[Token]) -> List[int]:
        # token_address => normalized rate
        return await self.prices.functions.getManyRatesToEthWithCustomConnectors(
            list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
            False, # use wrappers
            self.settings.connector_tokens_addrs,
            10 # threshold_filter
        ).call()

    @require_async_context
    async def get_prices(self, tokens: List[Token]) -> List[Price]:
        """Get prices for tokens in target stable token"""

        # batches starts as a list of lists 
        batches = await asyncio.gather(
            *map(
                # XX: lists are not cacheable, convert them to tuples so lru cache is happy
                lambda ts: self._get_prices(tuple(ts)),
                list(chunk(tokens, self.settings.price_batch_size)),
            )
        )
        return self.prepare_prices(tokens, sum(batches, []))

    @require_async_context
    async def get_pools(self) -> List[LiquidityPool]:
        pools, offset, limit = [], 0, self.settings.pool_page_size
        tokens = await self.get_all_tokens()
        prices = await self.get_prices(tokens)
        
        while True:
            pools_batch = await self.sugar.functions.all(limit, offset).call()
            pools += pools_batch
            if len(pools_batch) < limit: break
            else: offset += limit

        return self.prepare_pools(pools, tokens, prices)
    
    @require_async_context
    async def set_token_allowance(self, token: Token, addr: str, amount: int):
        ERC20_ABI = [{
            "name": "approve",
            "type": "function",
            "constant": False,
            "inputs": [{"name": "spender", "type": "address"}, {"name": "amount", "type": "uint256"}],
            "outputs": [{"name": "", "type": "bool"}]
        }]
        token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
        return await self.sign_and_send_tx(token_contract.functions.approve(addr, amount))

    @require_async_context
    async def check_token_allowance(self, token: Token, addr: str) -> int:
        ERC20_ABI = [{
            "name": "allowance",
            "type": "function",
            "constant": True,
            "inputs": [{"name": "owner", "type": "address"}, {"name": "spender", "type": "address"}],
            "outputs": [{"name": "", "type": "uint256"}]
        }]
        token_contract = self.web3.eth.contract(address=token.token_address, abi=ERC20_ABI)
        return await token_contract.functions.allowance(self.account.address, addr).call()

    @require_async_context
    async def sign_and_send_tx(self, tx, value: int = 0, wait: bool = True):
        print(f"sign_and_send_tx: {tx} with value: {value}")
        spender = self.account.address
        tx = await tx.build_transaction({ 'from': spender, 'value': value, 'nonce': await self.web3.eth.get_transaction_count(spender) })
        signed_tx = self.account.sign_transaction(tx)
        tx_hash = await self.web3.eth.send_raw_transaction(signed_tx.raw_transaction)
        return await self.web3.eth.wait_for_transaction_receipt(tx_hash) if wait else tx_hash
    
    @require_async_context
    async def deposit(self, deposit: Deposit, delay_in_minutes: float = 30, slippage: float = 0.01):
        amount_token0, pool, router_contract_addr = deposit.amount_token0, deposit.pool, self.settings.router_contract_addr
        print(f"gonna deposit {amount_token0} {pool.token0.symbol} into {pool.symbol} from {self.account.address}")
        [token0_amount, token1_amount, _] = await self.router.functions.quoteAddLiquidity(
            pool.token0.token_address,
            pool.token1.token_address,
            pool.is_stable,
            pool.factory,
            float_to_uint256(amount_token0, pool.token0.decimals),
            MAX_UINT256
        ).call()
        print(f"Quote: {pool.token0.symbol} {token0_amount / 10 ** pool.token0.decimals} -> {pool.token1.symbol} {token1_amount / 10 ** pool.token1.decimals}")

        # set up allowance for both tokens
        print(f"setting up allowance for {pool.token0.symbol}")
        await self.set_token_allowance(pool.token0, router_contract_addr, token0_amount)

        print(f"setting up allowance for {pool.token1.symbol}")
        await self.set_token_allowance(pool.token1, router_contract_addr, token1_amount)

        # check allowances
        token0_allowance = await self.check_token_allowance(pool.token0, router_contract_addr)
        token1_allowance = await self.check_token_allowance(pool.token1, router_contract_addr)

        print(f"allowances: {token0_allowance}, {token1_allowance}")

        # adding liquidity

        # if token 0 is native, use addLiquidityETH instead of standard addLiquidity
        if pool.token0.token_address == self.settings.wrapped_native_token_addr:
            params = [
                pool.token1.token_address,
                pool.is_stable,
                token1_amount,
                apply_slippage(token1_amount, slippage),
                apply_slippage(token0_amount, slippage),
                self.account.address,
                get_future_timestamp(delay_in_minutes)
            ]
            print(f"adding liquidity with params: {params}")
            return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token0_amount)
        
        # token 1 is native, use addLiquidityETH instead of standard addLiquidity
        if pool.token1.token_address == self.settings.wrapped_native_token_addr:
            params = [
                pool.token0.token_address,
                pool.is_stable,
                token0_amount,
                apply_slippage(token0_amount, slippage),
                apply_slippage(token1_amount, slippage),
                self.account.address,
                get_future_timestamp(delay_in_minutes)
            ]
            print(f"adding liquidity with params: {params}")
            return await self.sign_and_send_tx(self.router.functions.addLiquidityETH(*params), value=token1_amount)

        params = [
            pool.token0.token_address,
            pool.token1.token_address,
            pool.is_stable,
            token0_amount,
            token1_amount,
            apply_slippage(token0_amount, slippage),
            apply_slippage(token1_amount, slippage),
            self.account.address,
            get_future_timestamp(delay_in_minutes)
        ]

        print(f"adding liquidity with params: {params}")

        return await self.sign_and_send_tx(self.router.functions.addLiquidity(*params))

## Sync chain

In [ ]:
#| export

class Chain(CommonChain):
    web3: Web3
    sugar: Contract
    router: Contract

    def __enter__(self):
        """Sync context manager entry"""
        self._in_context = True
        self.web3 = Web3(HTTPProvider(self.settings.rpc_uri))
        self.sugar = self.web3.eth.contract(address=self.settings.sugar_contract_addr, abi=sugar)
        self.slipstream = self.web3.eth.contract(address=self.settings.slipstream_contract_addr, abi=slipstream)
        self.prices = self.web3.eth.contract(address=self.settings.price_oracle_contract_addr, abi=price_oracle)
        self.router = self.web3.eth.contract(address=self.settings.router_contract_addr, abi=router)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        """Sync context manager exit"""
        self._in_context = False
        return None
    
    @require_context
    def get_all_tokens(self, listed_only: bool = True) -> List[Token]:
        return self.prepare_tokens(self.sugar.functions.tokens(self.settings.pagination_limit, 0, ADDRESS_ZERO, []).call(), listed_only)
    
    # @cache_in_seconds(ORACLE_PRICES_CACHE_MINUTES * 60)
    def _get_prices(self, tokens: Tuple[Token]) -> List[int]:
        # token_address => normalized rate
        return self.prices.functions.getManyRatesToEthWithCustomConnectors(
            list(map(lambda t: t.wrapped_token_address or t.token_address, tokens)),
            False, # use wrappers
            self.settings.connector_tokens_addrs,
            10 # threshold_filter
        ).call()

    @require_context
    def get_prices(self, tokens: List[Token]) -> List[Price]:
        """Get prices for tokens in target stable token"""
        return self.prepare_prices(tokens, sum([self._get_prices(tuple(ts)) for ts in list(chunk(tokens, self.settings.price_batch_size))], []))
    
    @require_context
    def get_pools(self) -> List[LiquidityPool]:
        pools, offset, limit = [], 0, self.settings.pool_page_size
        tokens = self.get_all_tokens()
        prices = self.get_prices(tokens)
        
        while True:
            pools_batch = self.sugar.functions.all(limit, offset).call()
            pools += pools_batch
            if len(pools_batch) < limit: break
            else: offset += limit

        return self.prepare_pools(pools, tokens, prices)

## OP Chain

In [ ]:
#| export

class OPChainCommon():
    usdc: str = normalize_address("0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85")
    velo: str = normalize_address("0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db")

class AsyncOPChain(AsyncChain, OPChainCommon):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs))

class OPChain(Chain, OPChainCommon):
    def __init__(self, **kwargs): super().__init__(make_op_chain_settings(**kwargs))


## Base Chain

In [ ]:
#| export

class BaseChainCommon():
    usdc: str = normalize_address("0x833589fcd6edb6e08f4c7c32d4f71b54bda02913")
    aero: str = normalize_address("0x940181a94a35a4569e4529a3cdfb74e38fd98631")

class AsyncBaseChain(AsyncChain, BaseChainCommon):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs))

class BaseChain(Chain, BaseChainCommon):
    def __init__(self, **kwargs): super().__init__(make_base_chain_settings(**kwargs))

## Simnet versions of chains

Simnet URIs:

- OP: http://127.0.0.1:4444
- Base: http://127.0.0.1:4445

This assumes the following setup:

```
supersim fork  --l2.host=0.0.0.0 --l2.starting.port=4444 --chains=op,base
```

In [ ]:
#| export 

class AsyncOPChainSimnet(AsyncOPChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4444", **kwargs)

class AsyncBaseChainSimnet(AsyncBaseChain):
    def __init__(self,  **kwargs): super().__init__(rpc_uri="http://127.0.0.1:4445", **kwargs)

## Tests

Run tests using mainnets for reads and simnet for writes

Make sure simnet is running

In [ ]:
from fastcore.test import test_eq
import socket

host, port ='127.0.0.1', 4444
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex((host, port))
    test_eq(result, 0)
except socket.error as err:
    test_eq(err, None)
finally:
    sock.close()

Make sure PK is set for writes

In [ ]:
import os
from fastcore.test import test_ne
from dotenv import load_dotenv

load_dotenv()

test_ne(os.getenv("SUGAR_PK"), None)

Getting tokens. Make sure native token is included

In [ ]:
from fastcore.test import test_eq, test_ne

async with AsyncOPChain() as op:
    tokens = await op.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

with OPChain() as op_sync:
    tokens = op_sync.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

async with AsyncBaseChain() as base:
    tokens = await base.get_all_tokens()
    native_token = next(filter(lambda t: t.is_native, tokens), None)
    test_ne(native_token, None)
    test_eq(native_token.symbol, "ETH")

Check on tokens and pools

In [ ]:
async with AsyncOPChain() as chain:
    tokens = await chain.get_all_tokens()
    prices = await chain.get_prices(tokens)
    pools = await chain.get_pools()

    for p in prices[:5]: print(f"{p.token.symbol} price: ${p.price}")
    for p in pools[:5]: print(f"{p.symbol} pool: ${p.tvl}")

ETH price: $2555.97740169514
VELO price: $0.06548401387307201
RED price: $0.12068560936474831
USDC price: $1.0
WETH price: $2555.97740169514
vAMM-RED/VELO pool: $210631.24170408587
vAMM-USDC/VELO pool: $413264.02491533663
vAMM-WETH/VELO pool: $440805.90573255974
sAMM-alETH/frxETH pool: $800747.3586038395
vAMM-wstETH/LDO pool: $142363.84122667808


In [ ]:
with OPChain() as chain:
    tokens = chain.get_all_tokens()
    prices = chain.get_prices(tokens)
    pools = chain.get_pools()

    for p in prices[:5]: print(f"{p.token.symbol} price: ${p.price}")
    for p in pools[:5]: print(f"{p.symbol} pool: ${p.tvl}")

ETH price: $2556.001749060373
VELO price: $0.06548446198483665
RED price: $0.12068598520482875
USDC price: $1.0
WETH price: $2556.001749060373
vAMM-RED/VELO pool: $210631.2797342575
vAMM-USDC/VELO pool: $413264.0259370015
vAMM-WETH/VELO pool: $440805.90573255974
sAMM-alETH/frxETH pool: $800747.3586038395
vAMM-wstETH/LDO pool: $142363.84122667808


Basic deposit on OP

In [ ]:
async with AsyncOPChain() as chain:
    pools = await chain.get_pools()
    pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo, pools))
    async with AsyncOPChainSimnet() as supersim:
        print(f"supersim is {supersim}")
        # 0.02 USDC 
        await supersim.deposit(Deposit(pools[0], 0.02))

supersim is <__main__.AsyncOPChainSimnet object>
gonna deposit 0.02 USDC into vAMM-USDC/VELO from 0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA
Quote: USDC 0.02 -> VELO 0.3145059046008855
setting up allowance for USDC
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xa062aE8A9c5e11aaA026fc2670B0D65cCc8B2858', 20000)> with value: 0
setting up allowance for VELO
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xa062aE8A9c5e11aaA026fc2670B0D65cCc8B2858', 314505904600885506)> with value: 0
allowances: 20000, 314505904600885506
adding liquidity with params: ['0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85', '0x9560e827aF36c94D2Ac33a39bCE1Fe78631088Db', False, 20000, 314505904600885506, 19800, 311360845554876672, '0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA', 1740565948]
sign_and_send_tx: <Function addLiquidity(address,address,bool,uint256,uint256,uint256,uint256,address,uint256) bound to ('0x0b2C639c533813f4Aa9D7837CAf62653d097Ff85', '0x9560e827aF36c94D2Ac33a39bCE1Fe

ContractLogicError: ('execution reverted', 'no data')

Native -> non Native on OP

In [ ]:
## WIP
# async with OPChain() as op:
#     pools = await op.get_pools()
#     # find WETH/Velo pool
#     weth_velo_pool = next(filter(lambda p: p.token0.token_address == op.settings.wrapped_native_token_addr and p.token1.token_address == OPChain.velo, pools), None)
#     test_ne(weth_velo_pool, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(weth_velo_pool, 0.0001))


Basic deposit on Base

In [ ]:
async with AsyncBaseChain() as chain:
    pools = await chain.get_pools()
    pools = list(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.aero, pools))
    async with AsyncBaseChainSimnet() as supersim:
        # 0.02 USDC 
        await supersim.deposit(Deposit(pools[0], 0.01), slippage=0.05)

gonna deposit 0.01 USDC into vAMM-USDC/AERO from 0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA
Quote: USDC 0.01 -> AERO 0.01502349557500575
setting up allowance for USDC
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xcF77a3Ba9A5CA399B7c97c74d54e5b1Beb874E43', 10000)> with value: 0
setting up allowance for AERO
sign_and_send_tx: <Function approve(address,uint256) bound to ('0xcF77a3Ba9A5CA399B7c97c74d54e5b1Beb874E43', 15023495575005749)> with value: 0
allowances: 10000, 15023495575005749
adding liquidity with params: ['0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0x940181a94A35A4569E4529A3CDfB74e38FD98631', False, 10000, 15023495575005749, 9500, 14272320796255460, '0x1e7A6B63F98484514610A9F0D5b399d4F7a9b1dA', 1740519109]
sign_and_send_tx: <Function addLiquidity(address,address,bool,uint256,uint256,uint256,uint256,address,uint256) bound to ('0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913', '0x940181a94A35A4569E4529A3CDfB74e38FD98631', False, 10000, 15023495575005749, 9500, 1

CL on OP

In [ ]:
# WIP
# async with OPChain() as chain:
#     pools = await chain.get_pools()
#     # CL200-USDC/VELO
#     cl_usdc_velo = next(filter(lambda x: x.token0 and x.token0.token_address == chain.usdc and x.token1.token_address == chain.velo and x.is_cl, pools), None)
#     test_ne(cl_usdc_velo, None)
#     async with OPChainSimnet() as supersim:
#         await supersim.deposit(Deposit(cl_usdc_velo, 0.02))

In [ ]:
#| hide

import nbdev; nbdev.nbdev_export()